In [1]:
import pandas as pd
import ast
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from functions import parse_revision_history
from functions import classify_bypass_from_parsed
from functions import drop_clusters
from functions import compare_build_numbers
from functions import overlap_security_updates
from functions import eliminate_by_SU_and_RD

In [2]:
df = pd.read_csv('./2024_sample_data_update.csv')

In [3]:
df.head()

,Release date,Last Updated,CVE Number,CVE Number (Link),CVE Title,Vector String,CWE,revision,security_updates
0,"Jan 9, 2024","Jan 9, 2024",CVE-2024-20672,https://msrc.microsoft.com/update-guide/vulner...,.NET Denial of Service Vulnerability,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:N/A:H/E...,CWE-400: Uncontrolled Resource Consumption,"['1.0\nJan 9, 2024\nInformation published.\n']","[('.NET 7.0', '-', '7.0.15'), ('.NET 6.0', '-'..."
1,"Feb 13, 2024","Feb 13, 2024",CVE-2024-21386,https://msrc.microsoft.com/update-guide/vulner...,.NET Denial of Service Vulnerability,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:N/A:H/E...,CWE-400: Uncontrolled Resource Consumption,"['1.0\nFeb 13, 2024\nInformation published.\n']","[('Microsoft Visual Studio 2022 version 17.6',..."
2,"Feb 13, 2024","Feb 13, 2024",CVE-2024-21404,https://msrc.microsoft.com/update-guide/vulner...,.NET Denial of Service Vulnerability,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:N/A:H/E...,CWE-476: NULL Pointer Dereference,"['1.0\nFeb 13, 2024\nInformation published.\n']","[('Microsoft Visual Studio 2022 version 17.8',..."
3,"Jan 9, 2024","Feb 13, 2024",CVE-2024-21312,https://msrc.microsoft.com/update-guide/vulner...,.NET Framework Denial of Service Vulnerability,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:N/A:H/E...,CWE-20: Improper Input Validation,"['1.3\nFeb 13, 2024\nTo address a known issue ...","[('Microsoft .NET Framework 3.5', 'Windows Ser..."
4,"Apr 12, 2022","Jun 24, 2024",CVE-2022-26832,https://msrc.microsoft.com/update-guide/vulner...,.NET Framework Denial of Service Vulnerability,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:N/A:H/E...,NaN,"['4.1\nJun 24, 2024\nUpdated the build numbers...",[('Microsoft .NET Framework 3.5 AND 4.6.2/4.7/...


In [4]:
df.isna().sum()

Release date          0
Last Updated          0
CVE Number            0
CVE Number (Link)     0
CVE Title             1
Vector String        11
CWE                  20
revision              0
security_updates      0
dtype: int64

In [5]:
df['CVE Title'].fillna('', inplace=True)
df['Vector String'].fillna('', inplace=True)
df['CWE'].fillna('', inplace=True)

In [6]:
df.isna().sum()

Release date         0
Last Updated         0
CVE Number           0
CVE Number (Link)    0
CVE Title            0
Vector String        0
CWE                  0
revision             0
security_updates     0
dtype: int64

In [7]:
df['security_updates'] = df['security_updates'].apply(ast.literal_eval)
print(df['security_updates'].iloc[0])

[('.NET 7.0', '-', '7.0.15'), ('.NET 6.0', '-', '6.0.26')]


In [8]:
len(df['security_updates'].iloc[0])

2

In [9]:
df['revision_indication'] = df['revision'].apply(lambda x: classify_bypass_from_parsed(parse_revision_history(x)))

In [10]:
df['revision_indication']

0       unlikely
1       unlikely
2       unlikely
3       unlikely
4       unlikely
          ...   
996     unlikely
997     unlikely
998     unlikely
999     unlikely
1000    unlikely
Name: revision_indication, Length: 1001, dtype: object

In [11]:
#Cluster the rows with same or very similar CVE Titles


vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['CVE Title'])

similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

#set the threshold manually
threshold = 0.8

clusters = np.arange(len(similarity_matrix))

for i in range(len(similarity_matrix)):
    for j in range(i + 1, len(similarity_matrix)):
        if similarity_matrix[i, j] >= threshold:
            clusters[j] = clusters[i]

df['Cluster'] = clusters

In [12]:
len(df)

1001

In [13]:
df['Cluster'] = pd.factorize(df['Cluster'])[0]
cluster_lengths = df.groupby('Cluster').size()
print(cluster_lengths)

Cluster
0       3
1       2
2       1
3      12
4       4
       ..
346     1
347     2
348     3
349     1
350     1
Length: 351, dtype: int64


In [14]:
df = df.groupby('Cluster').apply(drop_clusters)
df = df.dropna().reset_index(drop=True)

In [15]:
len(df)

789

In [16]:
df['Cluster'] = pd.factorize(df['Cluster'])[0]
cluster_lengths = df.groupby('Cluster').size()
print(cluster_lengths)

Cluster
0       3
1       2
2      12
3       4
4       3
       ..
134     3
135     5
136     4
137     2
138     3
Length: 139, dtype: int64


In [17]:
df = df.groupby('Cluster').apply(eliminate_by_SU_and_RD)
df = df.reset_index(drop=True)

In [18]:
len(df)

515

In [19]:
df['Cluster'] = pd.factorize(df['Cluster'])[0]
cluster = df[df['Cluster'] == 0]['CVE Title']
print(cluster)

0    .NET and Visual Studio Denial of Service Vulne...
1    .NET, .NET Framework, and Visual Studio Denial...
2    .NET, .NET Framework, and Visual Studio Denial...
3    .NET, .NET Framework, and Visual Studio Denial...
4    .NET, .NET Framework, and Visual Studio Elevat...
5    .NET, .NET Framework, and Visual Studio Elevat...
6    .NET, .NET Framework, and Visual Studio Remote...
7    NET, .NET Framework, and Visual Studio Securit...
Name: CVE Title, dtype: object


In [20]:
cluster_lengths = df.groupby('Cluster').size()
print(cluster_lengths)

Cluster
0     8
1     2
2     3
3     4
4     2
     ..
83    2
84    3
85    5
86    4
87    3
Length: 88, dtype: int64


In [37]:
print(df[df['Cluster'] == 87]['CWE'])

512               CWE-122: Heap-based Buffer Overflow
513           CWE-190: Integer Overflow or Wraparound
514    CWE-250: Execution with Unnecessary Privileges
Name: CWE, dtype: object
